# GCcare NIST RAG API 테스트 노트북

이 노트북은 `FastAPI + LangServe`로 띄운 **/health**, **/rag** 엔드포인트를 간단히 테스트하기 위한 예시입니다.

## 0. 서버 실행 (터미널에서)

아나콘다 / venv를 활성화한 뒤, 프로젝트 루트(예: `.../gccare-langchain-rag-ai`)에서 아래 명령어를 **터미널에서** 실행해 FastAPI 서버를 먼저 띄워주세요.

```bash
python -m uvicorn api.serve_app:app --host 127.0.0.1 --port 8000 --reload
```

- 서버가 정상 실행되면: `http://127.0.0.1:8000/docs` 로도 접속 가능
- 이 노트북은 서버가 떠 있다는 가정하에 `http://127.0.0.1:8000` 으로 요청을 보냅니다.


In [1]:
import json
import textwrap
import requests

BASE_URL = "http://127.0.0.1:8000"

print("BASE_URL:", BASE_URL)

BASE_URL: http://127.0.0.1:8000


## 1. `/health` 엔드포인트 확인

FastAPI 서버가 제대로 떠 있는지 헬스 체크를 해봅니다.


In [3]:
health_res = requests.get(f"{BASE_URL}/health")
print("Status code:", health_res.status_code)
print("Response JSON:")
print(health_res.json())

Status code: 200
Response JSON:
{'status': 'ok'}


## 2. RAG `/rag/invoke` 기본 헬퍼 함수

LangServe가 마운트한 `/rag/invoke` 엔드포인트에 POST 요청을 보내는 헬퍼 함수를 만듭니다.

- body 형식: `{"input": "<질문 문자열>"}`


In [4]:
def ask_rag(question: str):
    """RAG 체인(/rag/invoke)에 질문을 던지고 결과를 반환하는 헬퍼 함수."""
    url = f"{BASE_URL}/rag/invoke"
    payload = {"input": question}
    print("\n=== 질문 =============================")
    print(question)
    print("=====================================")
    
    res = requests.post(url, json=payload)
    print("Status code:", res.status_code)
    try:
        data = res.json()
        print("\n원시 응답(JSON):")
        print(json.dumps(data, ensure_ascii=False, indent=2))
    except Exception:
        print("\n응답 본문(text):")
        print(res.text)
        
    return res

## 3. 기본 테스트 질문 1 — NIST AI RMF 목적

가장 기본적인 질문으로, NIST AI Risk Management Framework의 목적에 대해 물어봅니다.


In [5]:
_ = ask_rag("NIST AI Risk Management Framework의 핵심 목적이 뭐야?")


=== 질문 =============================
NIST AI Risk Management Framework의 핵심 목적이 뭐야?
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI Risk Management Framework의 핵심 목적은 인공지능 시스템의 위험을 식별, 평가, 관리하여 안전하고 책임 있는 AI 개발과 사용을 촉진하는 것입니다.",
  "metadata": {
    "run_id": "cee7d362-b706-4de7-8474-f02b840cbbfe",
    "feedback_tokens": []
  }
}


## 4. 테스트 질문 2 — Governance 관련

거버넌스(거버넌스 기능, 역할 등)에 초점을 맞춰서 질문해 봅니다.


In [6]:
_ = ask_rag("NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할을 정리해줘.")


=== 질문 =============================
NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할을 정리해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할은 다음과 같습니다:\n\n1. 정책 개발: AI 시스템의 설계, 개발 및 운영에 필요한 정책과 기준을 수립합니다.\n2. 책임 및 권한 설정: AI 관련 결정에 대한 책임과 권한을 명확히 합니다.\n3. 리스크 관리: AI 시스템의 리스크를 평가하고 관리하는 프로세스를 수립합니다.\n4. 이해관계자 참여: 다양한 이해관계자와의 소통과 협력을 통해 거버넌스를 강화합니다.\n5. 지속적인 모니터링: AI 시스템의 성과와 영향을 지속적으로 모니터링하고 평가합니다.\n\n이러한 활동들은 AI 시스템이 윤리적이고 투명하게 운영되도록 하는 데 기여합니다.",
  "metadata": {
    "run_id": "e15f66e1-0deb-4ff5-ac5a-d26f91215491",
    "feedback_tokens": []
  }
}


## 5. 테스트 질문 3 — 라이프사이클 관점 질문

NIST AI RMF의 구조(예: Map, Measure, Manage, Govern 등)에 초점.


In [7]:
_ = ask_rag("NIST AI RMF에서 Map, Measure, Manage, Govern 단계가 각각 무엇을 의미하는지 요약해줘.")


=== 질문 =============================
NIST AI RMF에서 Map, Measure, Manage, Govern 단계가 각각 무엇을 의미하는지 요약해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF에서 각 단계의 의미는 다음과 같습니다:\n\n1. **Map (매핑)**: AI 시스템의 맥락을 이해하고, 관련된 위험과 기회를 식별하는 단계입니다.\n2. **Measure (측정)**: AI 시스템의 성능과 위험을 평가하고, 해당 시스템이 규정과 기대에 부합하는지를 측정합니다.\n3. **Manage (관리)**: 식별된 위험을 효과적으로 관리하고 대응하는 방안을 마련하는 단계입니다.\n4. **Govern (거버넌스)**: AI 시스템의 개발과 운영이 법적, 윤리적 기준 및 내부 정책에 부합하도록 보장하는 단계입니다.",
  "metadata": {
    "run_id": "4378e8c5-1da9-434e-a0b2-1b8030a4b1b5",
    "feedback_tokens": []
  }
}


## 6. 테스트 질문 4 — 실무 적용 관점 질문

실제로 조직이 AI 리스크 관리를 도입할 때의 적용 관점 질문입니다.


In [8]:
_ = ask_rag(
    "중견 교육기업이 NIST AI RMF를 참고해서 생성형 AI 서비스 리스크 관리를 하려면, 우선적으로 어떤 영역부터 점검하는 것이 좋을까?"
)


=== 질문 =============================
중견 교육기업이 NIST AI RMF를 참고해서 생성형 AI 서비스 리스크 관리를 하려면, 우선적으로 어떤 영역부터 점검하는 것이 좋을까?
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF(Artificial Intelligence Risk Management Framework)를 참고하여 중견 교육기업이 생성형 AI 서비스 리스크 관리를 하려면, 우선적으로 다음 영역을 점검하는 것이 좋습니다:\n\n1. **리더십 및 거버넌스**: AI 서비스에 대한 조직의 리더십과 거버넌스 구조를 점검하여 책임과 의사 결정 프로세스를 명확히 합니다.\n2. **데이터 관리**: 사용되는 데이터의 품질, 편향, 개인정보 보호 관련 규정을 준수하는지 점검합니다.\n3. **모델 개발 및 테스트**: 생성형 AI 모델의 개발 과정과 테스트 절차를 검토하여 신뢰성과 안전성을 보장합니다.\n4. **위험 식별 및 평가**: AI 서비스와 관련된 잠재적 리스크를 식별하고 평가합니다.\n\n이러한 영역을 점검하며, 리스크 관리 프레임워크를 체계적으로 구축하는 것이 중요합니다.",
  "metadata": {
    "run_id": "41b0505e-0dcf-48eb-b0b7-d4a691893934",
    "feedback_tokens": []
  }
}


## 7. 테스트 질문 5 — 문서에 없을 법한 질문 (Out-of-Scope)

RAG가 **컨텍스트에 없는 내용은 모른다고 답하는지** 확인하기 위해 일부러 문서와 관계없는 질문을 던져봅니다.


In [9]:
_ = ask_rag("NIST AI RMF 문서에서 BTS 콘서트 일정에 대해 뭐라고 말하고 있어?")


=== 질문 =============================
NIST AI RMF 문서에서 BTS 콘서트 일정에 대해 뭐라고 말하고 있어?
Status code: 200

원시 응답(JSON):
{
  "output": "모릅니다.",
  "metadata": {
    "run_id": "65897a30-8b08-4fc0-ad6e-5f774df55a7e",
    "feedback_tokens": []
  }
}


## 8. 자유 질문 셀

원하는 질문으로 직접 RAG 호출을 해볼 수 있는 셀입니다. `question` 변수만 바꿔서 실행하세요.


In [10]:
question = "AI 시스템의 신뢰성과 투명성을 동시에 높이기 위해 NIST AI RMF에서 제안하는 원칙들을 간단히 정리해줘."
_ = ask_rag(question)


=== 질문 =============================
AI 시스템의 신뢰성과 투명성을 동시에 높이기 위해 NIST AI RMF에서 제안하는 원칙들을 간단히 정리해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF(인공지능 위험 관리 프레임워크)에서 제안하는 원칙들은 다음과 같습니다:\n\n1. **신뢰성**: AI 시스템이 일관되게 예측하고 결과를 제공할 수 있도록 할 것.\n2. **투명성**: AI 시스템의 동작과 의사결정을 이해할 수 있는 방법을 제공할 것.\n3. **공정성**: 차별이나 편견 없이 모든 사용자에게 공정하게 작동하도록 설계할 것.\n4. **개인정보 보호**: 사용자 데이터의 보안과 개인정보 보호를 보장할 것.\n5. **사용자 참여**: 사용자와 이해관계자의 피드백을 적극적으로 반영하여 시스템을 개선할 것.\n6. **책임성**: AI 시스템의 결과에 대한 책임을 명확히 하고, 문제 발생 시 책임을 질 수 있는 구조를 마련할 것.\n\n이러한 원칙들은 AI 시스템의 신뢰성과 투명성을 높이는 데 기여합니다.",
  "metadata": {
    "run_id": "c9b45517-f858-4cef-b164-09e88a0f30ba",
    "feedback_tokens": []
  }
}
